In [5]:
!wget -qq https://github.com/shenasa-ai/persian-book-recommender-dataset/raw/main/data.zip
!wget -qq https://github.com/Alireza-Akhavan/temp/raw/main/x_tree.csv
!wget -qq https://raw.githubusercontent.com/Alireza-Akhavan/temp/main/x_tag.csv

In [1]:
!unzip -qq data.zip

replace sales.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [2]:
import random
import pandas as pd
import numpy as np

sales_data = pd.read_csv('sales.csv')
goods_df = pd.read_csv('goods.csv')
tag_df = pd.read_csv('x_tag.csv')
tree_df = pd.read_csv('x_tree.csv')
customer_df = pd.read_csv('customer.csv')

## encode catgory column

In [3]:
goods_df.head()

,good_id,brand,author,translator,catgory,tag
0,162,96,115,NaN,153,1|1884|1965|3728|48864
1,163,96,115,NaN,66,1|434|438|441|454|518|552|727|1355|1480|1884|1...
2,165,96,1724,NaN,139|141,1|1884|1965|3728|48864
3,166,96,2043,NaN,118,1|1884|1965|3728|48864
4,167,96,1829,NaN,139|140,1|1884|1965|3728|48864


In [4]:
catgory = goods_df['catgory'].str.split('|').apply(lambda x: list(map(int, x)) if isinstance(x, list) else []).tolist()
catgory[:10]

[[153],
 [66],
 [139, 141],
 [118],
 [139, 140],
 [130],
 [120],
 [167],
 [64, 73],
 [62, 65, 66]]

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
catgory_encoded = pd.DataFrame(mlb.fit_transform(catgory), columns=mlb.classes_, index=goods_df.index)

# Concatenate the encoded catgory DataFrame with the original DataFrame
good_id_to_catgory_df = pd.concat([goods_df['good_id'], catgory_encoded], axis=1)
good_id_to_catgory_df

,good_id,29,31,32,35,37,38,40,41,42,...,170,171,173,178,189,190,191,192,193,197
0,162,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,163,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,165,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,166,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,167,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57088,105771,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57089,105772,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57090,105773,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57091,105774,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## encode author column

In [5]:
goods_df[goods_df['author'].str.contains('\|', na=False)]

,good_id,brand,author,translator,catgory,tag
84,250,1612,11838|25571|33469,161,60,1876|1884|3728|10709
155,324,1546,122|11641,NaN,40,1868|1876|1884|1984|3728|10709|19664
189,360,1546,122|1967,NaN,35|132,1868|1876|1884|1984|3728|10709|19664
195,366,1546,1667|15511,NaN,60|70|130,1868|1876|1884|1984|3728|10709
208,381,1582,2099|2636,NaN,148,10709
...,...,...,...,...,...,...
57015,105655,2155,68826|68827,105652|105653|105654,29,1868|1873|1882|1899|1965|1971|1984|3728|10709|...
57017,105658,2155,60662|60754|105657,NaN,29,1868|1873|1882|1899|1965|1971|1984|3728|10709|...
57049,105713,73,105711|105712,NaN,29,1868|1869|1878|1880|1884|1899|1965|1971|1980|1...
57050,105714,73,105711|105712,NaN,29,NaN


In [6]:
good_id2author = goods_df[['good_id','author']]
good_id2author = good_id2author.assign(author=good_id2author['author'].str.split('|').str[0])
good_id2author

,good_id,author
0,162,115
1,163,115
2,165,1724
3,166,2043
4,167,1829
...,...,...
57088,105771,103
57089,105772,103
57090,105773,103
57091,105774,103


In [7]:
good_id2author[good_id2author['author']==0]

,good_id,author


In [8]:
author_counts = good_id2author['author'].value_counts()

In [9]:
# Identify author codes with more than 1 occurrences
selected_author_codes = author_counts[author_counts > 1].index

In [10]:
good_id2author.loc[~good_id2author['author'].isin(selected_author_codes), 'author'] =0

In [11]:
good_id2author[good_id2author['author']==0]

,good_id,author
12,175,0
15,178,0
20,183,0
24,187,0
25,188,0
...,...,...
57062,105738,0
57063,105740,0
57065,105743,0
57066,105746,0


In [12]:
mask = good_id2author['author'] == 0
good_id2author.loc[~mask, 'author'] = pd.factorize(good_id2author.loc[~mask, 'author'])[0]+1

In [13]:
len(np.unique(good_id2author['author']))

6124

## customers

In [14]:
customer_df

,customer_id,gender
0,1,1
1,2,2
2,3,1
3,4,1
4,5,1
...,...,...
204913,205813,1
204914,205814,1
204915,205815,2
204916,205816,1


In [15]:
customer_df['gender'] = pd.factorize(customer_df['gender'])[0]
customer_df

,customer_id,gender
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0
...,...,...
204913,205813,0
204914,205814,0
204915,205815,1
204916,205816,0


## purchased_data

In [19]:
purchased_data = sales_data[['member_id', 'good_id']].drop_duplicates()
purchased_data

,member_id,good_id
0,5,15347
1,8,2833
2,8,8714
3,9,15672
4,12,15001
...,...,...
525082,205808,97125
525083,205809,35191
525084,205809,37009
525085,205809,77359


## **member data**




In [25]:
customer_df.rename(columns={'customer_id': 'member_id'}, inplace=True)
customer_df['gender'] = pd.factorize(customer_df['gender'])[0]
customer_df

,member_id,gender
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0
...,...,...
204913,205813,0
204914,205814,0
204915,205815,1
204916,205816,0


## good data

In [26]:
good_data = pd.merge(purchased_data, good_id2author, on='good_id', how='left')
good_data = good_data.drop('member_id', axis=1)

In [27]:
good_data.head()

,good_id,author
0,15347,601
1,2833,399
2,8714,0
3,15672,NaN
4,15001,388


In [28]:
good_data[good_data['author'].isna()]=0
good_data.head()

,good_id,author
0,15347,601
1,2833,399
2,8714,0
3,0,0
4,15001,388


In [29]:
good_data = pd.merge(good_data, good_id_to_catgory_df, on='good_id', how='left')
good_data

,good_id,author,29,31,32,35,37,38,40,41,...,170,171,173,178,189,190,191,192,193,197
0,15347,601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2833,399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8714,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15001,388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511134,97125,4730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511135,35191,1664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511136,37009,1499,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511137,77359,2733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
good_data.fillna(0, inplace=True)
good_data

,good_id,author,29,31,32,35,37,38,40,41,...,170,171,173,178,189,190,191,192,193,197
0,15347,601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2833,399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8714,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15001,388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511134,97125,4730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511135,35191,1664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511136,37009,1499,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511137,77359,2733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## create pairs

In [31]:
new_id, old_id = pd.factorize(purchased_data['member_id'])
member_id_mapping = dict(zip(old_id, new_id))
new_id, old_id = pd.factorize(purchased_data['good_id'])
good_id_mapping = dict(zip(old_id, new_id))

In [32]:
inverted_member_id_mapping = dict(zip(member_id_mapping.values(), member_id_mapping.keys()))
inverted_good_id_mapping = dict(zip(good_id_mapping.values(), good_id_mapping.keys()))

In [34]:
purchased_data['member_id'] = purchased_data['member_id'].map(member_id_mapping)
purchased_data['good_id'] = purchased_data['good_id'].map(good_id_mapping)

customer_df['member_id'] = customer_df['member_id'].map(member_id_mapping)
good_data['good_id'] = good_data['good_id'].map(good_id_mapping)

In [52]:
purchased_set = set(zip(sales_data['member_id'], sales_data['good_id']))
len(purchased_set)

511139

In [53]:
unique_members = purchased_data['member_id'].unique()
unique_goods = purchased_data['good_id'].unique()
n_unique_members = len(unique_members)
n_unique_goods = len(unique_goods)

In [54]:
not_purchased_set = set()

while len(not_purchased_set)!=len(purchased_set):
    random_member_id = random.choice(unique_members)
    random_good_id = random.choice(unique_goods)
    random_tuple = (random_member_id, random_good_id)
    if random_tuple not in purchased_set:
        not_purchased_set.add(random_tuple)

In [55]:
columns = ['member_id', 'good_id']
purchased_set = pd.DataFrame(purchased_set, columns=columns)
purchased_set['lbl'] = 1
not_purchased_df = pd.DataFrame(not_purchased_set, columns=columns)
not_purchased_df['lbl'] = 0

In [56]:
all_data = pd.concat([purchased_set, not_purchased_df], ignore_index=True)
all_data

,member_id,good_id,lbl
0,51579,30977,1
1,149602,97119,1
2,61753,63375,1
3,179428,52953,1
4,56460,18613,1
...,...,...,...
1022273,2813,10653,0
1022274,19735,6816,0
1022275,13171,3733,0
1022276,903,12223,0


In [ ]:
all_data = pd.merge(all_data, customer_df, on='member_id', how='left')
all_data

In [108]:
from sklearn.model_selection import train_test_split

df_train, df_test= train_test_split(all_data, test_size=0.2, random_state=42)

print(df_train.shape)
print(df_test.shape)

(817822, 3)
(204456, 3)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_unique_members, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-User")(user_embedding)

product_input = Input(shape=[1], name="Product-Input")
product_embedding = Embedding(n_unique_goods, 5, name="Product-Embedding")(product_input)
product_vec = Flatten(name="Flatten-Product")(product_embedding)

prod = Dot(name="Dot-Product", axes=1)([product_vec, user_vec])
model = Model([user_input, product_input], prod)
model.compile('adam', tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])



In [ ]:
history = model.fit([X_train[:,0],X_train[:,1]], y_train, epochs=10, batch_size=1024)

Epoch 1/10
583/583 [==============================] - 15s 25ms/step - loss: 0.5740 - accuracy: 0.7231 - val_loss: 0.4941 - val_accuracy: 0.8434
Epoch 2/10
583/583 [==============================] - 4s 7ms/step - loss: 0.4099 - accuracy: 0.8633 - val_loss: 0.3913 - val_accuracy: 0.8545
Epoch 3/10
583/583 [==============================] - 4s 7ms/step - loss: 0.2867 - accuracy: 0.8857 - val_loss: 0.3418 - val_accuracy: 0.8626
Epoch 4/10
583/583 [==============================] - 4s 7ms/step - loss: 0.2373 - accuracy: 0.8971 - val_loss: 0.3455 - val_accuracy: 0.8614
Epoch 5/10
583/583 [==============================] - 3s 5ms/step - loss: 0.2111 - accuracy: 0.9064 - val_loss: 0.3639 - val_accuracy: 0.8579
Epoch 6/10
583/583 [==============================] - 3s 5ms/step - loss: 0.1917 - accuracy: 0.9147 - val_loss: 0.3889 - val_accuracy: 0.8546
Epoch 7/10
583/583 [==============================] - 3s 5ms/step - loss: 0.1766 - accuracy: 0.9215 - val_loss: 0.4185 - val_accuracy: 0.8506
Epoc